<html>
<div style="width:100%">
  <div style="width:90%; float:left; background:white; margin:10px">
    <img style ="width:100%"src ="https://raw.githubusercontent.com/publicunpublic/QCMM_Jupyter/main/full.svg">
    <div style="margin-top:2.5%;position:absolute;background-color:#00000;width:6%;height:0.3%"></div>
  </div>
  </div>


<html>
<center>
<br>
<h1 style="color:#E6610F;"> Create ReactionDataset to compute TS and reaction energies </h1>
<br><br>
</center>

In [ ]:
import qcportal as ptl

## Define variable name

In [ ]:
client_address = "152.74.10.245:7778"
username = None
password = None 

# Collections with monomers
mon_opt_col_name = 'monomers'
monomer_name = 'd-lactide'
init_opt_col_name = 'initiators'
initiator_name = 'salen-OMe'

# Collections with the reactant products and intermediates
ri_opt_col_name = "salen_d-lac_rpi"
## Collections with the tranistion states
ts_opt_col_name = "salen_d-lac_ts_from_neb"
## Name of the ReactionDataset collection
name_rx = "salen_d-lac_rxn_b3lyp-d3_defs-svp"

# Names of reactants, transistion state and intermediate/product 
reac = 'r'
ts = 'ts1'
interm = 'i1'

In [ ]:
client = ptl.FractalClient(address=client_address, username = username, password = password, verify=False)

## A) Create ReactionDataset (If it does not exist yet)

In [ ]:
#ds_rx = ptl.collections.ReactionDataset(name_rx, ds_type="rxn", client=client, default_program="psi4")
#ds_rx.save()

## B) Call the OptimizationDataset with the optimized states

In [ ]:
ri_opt   =  client.get_collection("OptimizationDataset", ri_opt_col_name)
ts_opt   =  client.get_collection("OptimizationDataset", ts_opt_col_name)
mon_opt  =  client.get_collection("OptimizationDataset", mon_opt_col_name)
init_opt =  client.get_collection("OptimizationDataset", init_opt_col_name)

In [ ]:
ri_opt.status(collapse = False)

In [ ]:
ds_rx = client.get_collection("ReactionDataset", name_rx)

## C) Create the stoichiometry 

In [ ]:
lot = "b3lyp-d3_def2-svp"

mol_ids = {}
recs = ri_opt.data.records
paths = []

for path_key in recs.keys():
    if not ts in path_key:
        continue
    paths.append(path_key.split("_")[1])
    
paths = list(set(paths))
print(paths)

mon_rec = mon_opt.get_record(monomer_name, specification = lot)
opt_mol_nom = mon_rec.get_final_molecule()

initiator_rec = initiator_opt.get_record(intiator_name, specification = lot)
opt_mol_initiator = initiator_rec.get_final_molecule()


for p in paths:
    reac_path_name = ts+'_'+p+'_'+reac
    int_path_name =  ts+'_'+p+'_'+interm
    ts_path_name =  ts+'_'+p
    rx_entry_name =  reac+'_'+ts+'_'+interm+'_'+p

    reac_rec = ri_opt.get_record(reac_path_name, specification = lot)
    int_rec = ri_opt.get_record(int_path_name, specification = lot)
    ts_rec  = ts_opt.get_record(ts_path_name, specification = lot)
   

    if (reac_rec.status  == "ERROR" or ts_rec.status  == "ERROR" or int_rec.status  == "ERROR"):
        print("Reaction in path {} ended with an error.".format(path_key))
        continue
    
    opt_mol_int = int_rec.get_final_molecule()
    opt_mol_reac = reac_rec.get_final_molecule()
    opt_mol_ts = ts_rec.get_final_molecule()



    be_cal = {
            "default":[(opt_mol_int,1.0),(opt_mol_reac,-1.0)],
            "rx_energy" :[(opt_mol_int,1.0),(opt_mol_reac,-1.0)],
            "ts_energy" :[(opt_mol_ts,1.0),(opt_mol_reac,-1.0)],
            "rev_ts_energy" :[(opt_mol_ts,1.0),(opt_mol_int,-1.0)],
            "inter_energy" :[(opt_mol_mon,-1.0),(opt_mol_initiator,-1.0), (opt_mol_reac,1.0)],
            }

    ds_rx.add_rxn(rx_entry_name, be_cal)
    ds_rx.save()

## D) Check Records 

In [ ]:
ds_rx.data.records

## E) Compute energies

In [ ]:
## Define levels of theory to be computed

elot = ["B3LYP-D3BJ", "M06-L" , "PBE0-D3BJ", 'M11-L']

In [ ]:
# Send computations to server
for i in elot:
    c =  ds_rx.compute(i, "def2-tzvp" , stoich="default", tag='rx_comp')
    c1 = ds_rx.compute(i, "def2-tzvp" , stoich="ts_energy", tag='rx_comp')
    c2 = ds_rx.compute(i, "def2-tzvp" , stoich="rev_ts_energy", tag='rx_comp')

    print(c)
    print(c1)
    print(c2)
c_list = [c, c1, c2]

##  D) Check stoichiometry records 

In [ ]:
ds_rx = client.get_collection("ReactionDataset", name_rx)
stoich = 'default'

rq_method = []

for m in elot:
    if 'D3' in m:
        rq_method.append(m.split('-')[0])
    else:
        rq_method.append(m)


for m in rq_method:
    print("Records for {}".format(m))
    print(ds_rx.get_records(method=m, stoich=stoich))

## E) Get Reaction Energies

In [ ]:
ds_rx.get_values(method=elot, stoich='default')

In [ ]:
ds_rx.get_values(method=elot, stoich='ts_energy')

## Restart Jobs

To restart jobs you need to run compute jobs again to retrive the job ids

In [ ]:
stoichs = ds_rx.data.records[0].stoichiometry.keys()

rq_method = []

for m in elot:
    if 'D3' in m:
        rq_method.append(m.split('-')[0])
    else:
        rq_method.append(m)

for m in rq_method:
    for st in stoichs:
        all_rec =  ds_rx.get_records(method=m, stoich=st)['record']
        for r in all_rec:
            print(m, r, st)
            idd = r.dict()['id']
            if r.dict()['status'] == 'ERROR':
                restart = client.modify_tasks("restart", base_result = idd)
                print(restart)

In [ ]:
restart = client.modify_tasks("restart", base_result = 39144)
restart

##  Delete Entries

In [ ]:
#del ds_rx.data.records[0]
#ds_rx.save()